# Create Structural Elements using Autodesk Automation API

This notebook demonstrates how to create structural beam elements in Revit models using the Autodesk Platform Services (APS) Design Automation API.

**Important:** Unlike other examples, this workflow uploads a JSON configuration file to OSS (Object Storage Service) instead of embedding it directly in the work item payload. This approach is necessary because work item payloads are limited to 60KB.

**Reference:** https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step7-post-workitem/#additional-notes

By uploading the JSON file to storage and passing it as an input parameter, we can work with larger configuration files without hitting payload size limits.


## Step 1: Setup and Import Libraries

Import all necessary libraries and modules.

In [6]:
import os
import json
import logging
from pathlib import Path
from dotenv import load_dotenv

from aps_automation_sdk.classes import (
    Activity,
    ActivityInputParameter,
    ActivityOutputParameter,
    AppBundle,
    WorkItem
)

from aps_automation_sdk.utils import (
    delete_activity,
    delete_appbundle,
    get_token,
    set_nickname
)

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

## Step 2: Load Environment Variables and Configure Authentication

Load credentials from environment variables and get an authentication token.

**Prerequisites:**
1. Create an APS application to get your credentials. Follow these tutorials:
   - Create App: https://aps.autodesk.com/en/docs/oauth/v2/tutorials/create-app
   - Design Automation Setup: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step2-create-forge-app/

2. Create a `.env` file in the project root with your credentials (see `.env.sample` for reference)

**Note on Nicknames:** The nickname must be unique across all APS apps globally. If you encounter errors, try using alphanumeric unique values. Choose a distinctive nickname to avoid conflicts.

In [7]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")

# Get authentication token
token = get_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

# Set nickname. If the app already has a nickname, the previous one will be returned.
nickname = set_nickname(token, "myUniqueNickNameHere")

print(f"Authentication successful. Nickname: {nickname}")

Authentication successful. Nickname: viktortest


## Step 3: Define Project Constants

Set up all the constants needed for the structural elements creation workflow.

**Note:** The input Revit file, JSON configuration, and compiled app bundle can be found in `examples\03_structural_elements\files`

In [8]:
# Define constants
app_bundle_name = "CreateBeamElements"
activity_name = "CreateBeamElementsActivity"
alias = "prod"
bucket_key = "vkt003_bucket_beam_elements"
zip_path = Path.cwd() / "files" / "MyRevitAddin.bundle.zip"

# Create full aliases
appbundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
activity_full_alias = f"{nickname}.{activity_name}+{alias}"

print(f"App Bundle: {appbundle_full_alias}")
print(f"Activity: {activity_full_alias}")

App Bundle: viktortest.CreateBeamElements+prod
Activity: viktortest.CreateBeamElementsActivity+prod


## Step 4: Create and Deploy App Bundle

Register and upload the app bundle that contains the structural elements creation logic.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step4-publish-appbundle/

In [9]:
bundle = AppBundle(
    appBundleId=app_bundle_name,
    engine="Autodesk.Revit+2024",
    alias=alias,
    zip_path=str(zip_path),
    description="Create Structural Elements",
)

bundle.deploy(token)
print("App bundle deployed successfully!")

App bundle deployed successfully!


## Step 5: Define Activity Parameters

Create input and output parameters for the activity.

**Note:** This example includes a JSON file as an input parameter. The JSON will be uploaded to OSS storage rather than embedded in the work item payload to avoid the 60KB size limitation.

**Reference:** https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/#step-1-create-a-new-activity

In [10]:
# Input: Revit file
input_revit = ActivityInputParameter(
    name="rvtFile",
    localName="input.rvt",
    verb="get",
    description="Input Revit File",
    required=True,
    is_engine_input=True,
    bucketKey=bucket_key,
    objectKey="input.rvt",
)

# Output: Zipped IFC files
output_file = ActivityOutputParameter(
    name="result",
    localName="result.rvt",
    verb="put",
    description="Output revit file",
    bucketKey=bucket_key,
    objectKey="result.rvt",
)

# Input: JSON settings
input_json = ActivityInputParameter(
    name="jsonFile",
    localName="structure.json",
    verb="get",
    description="Input Revit File",
    required=True,
    bucketKey=bucket_key,
    objectKey="structure.json",
)
print("Activity parameters defined successfully!")

Activity parameters defined successfully!


## Step 6: Create and Deploy Activity

Create the activity that links the app bundle with the parameters.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/

In [11]:
activity = Activity(
    id=activity_name,
    parameters=[input_revit, output_file, input_json],
    engine="Autodesk.Revit+2024",
    appbundle_full_name=appbundle_full_alias,
    description="Create Beam Elements from a jsons",
    alias=alias,
)

# Set the Revit command
activity.set_revit_command_line()
activity.deploy(token=token)
print("Activity deployed successfully!")

Activity deployed successfully!


## Step 7: Upload Input Files

Upload the Revit file and JSON configuration file to Object Storage Service (OSS).

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step6-prepare-cloud-storage/

In [12]:
input_rvt_path = Path.cwd() / "files" / "revit_input.rvt"
input_revit.upload_file_to_oss(file_path=str(input_rvt_path), token=token)
print(f"Input Revit file uploaded: {input_rvt_path.name}")

Input Revit file uploaded: revit_input.rvt


In [13]:
input_json_path = Path.cwd() / "files" / "structure.json"
input_json.upload_file_to_oss(file_path=str(input_json_path), token=token)
print(f"Input Revit file uploaded: {input_json_path.name}")

Input Revit file uploaded: structure.json


## Step 8: Create and Execute Work Item

Create a work item with all parameters and execute it.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step7-post-workitem/

In [14]:
work_item = WorkItem(
    parameters=[input_revit, output_file, input_json],
    activity_full_alias=activity_full_alias
)

print("Work item created. Starting execution...")

Work item created. Starting execution...


## Step 10: Poll Work Item Status

Monitor the work item execution until completion (max 10 minutes).

In [15]:
status_resp = work_item.execute(token=token, max_wait=600, interval=10)
last_status = status_resp.get("status", "")

print(f"Work item completed with status: {last_status}")
print(f"Full response: {json.dumps(status_resp, indent=2)}")

2025-11-02 23:44:35,269 INFO Polling work item status, id=eee584beebaa45b1b6941c04aa965da6
2025-11-02 23:44:36,075 INFO [  0s] status=inprogress report_url=None
2025-11-02 23:44:46,954 INFO [ 10s] status=inprogress report_url=None
2025-11-02 23:44:57,822 INFO [ 20s] status=inprogress report_url=None
2025-11-02 23:45:08,669 INFO [ 30s] status=inprogress report_url=None
2025-11-02 23:45:19,490 INFO [ 40s] status=inprogress report_url=None
2025-11-02 23:45:30,287 INFO [ 50s] status=success report_url=https://dasprod-store.s3.amazonaws.com/workItem/viktortest/eee584beebaa45b1b6941c04aa965da6/report?X-Amz-Expires=3600&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEI3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIC%2BXQo%2BgtFVDdFRG9oLR16zHQrGr9Z5CdEAPKcJR%2BS3nAiEA3Zi8pCJKDPdISWZ8Nco8%2FYBLibR9r8%2Fgnvb6HPN9ngsqgAMIVRAEGgwyMjA0NzMxNTIzMTAiDK36jsM%2Fv36ae5vPZirdAuzpLf3X8F9MiBld2KkcCYKm4Pgo8VNHyZdImqPJGrzdW8gto4U2kABz%2F4oAnX6xFWZF3zwvB%2FuBF8%2B4PrUmg9a9Eh8m8LgGfJuZB9wOAu6rAhciu3Je8J%2BkWL5UcZ

Work item completed with status: success
Full response: {
  "status": "success",
  "reportUrl": "https://dasprod-store.s3.amazonaws.com/workItem/viktortest/eee584beebaa45b1b6941c04aa965da6/report?X-Amz-Expires=3600&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEI3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIC%2BXQo%2BgtFVDdFRG9oLR16zHQrGr9Z5CdEAPKcJR%2BS3nAiEA3Zi8pCJKDPdISWZ8Nco8%2FYBLibR9r8%2Fgnvb6HPN9ngsqgAMIVRAEGgwyMjA0NzMxNTIzMTAiDK36jsM%2Fv36ae5vPZirdAuzpLf3X8F9MiBld2KkcCYKm4Pgo8VNHyZdImqPJGrzdW8gto4U2kABz%2F4oAnX6xFWZF3zwvB%2FuBF8%2B4PrUmg9a9Eh8m8LgGfJuZB9wOAu6rAhciu3Je8J%2BkWL5UcZUhgXRQMlTpJKpal2P%2FJePpu23I5aR%2FBjBHpXNL9f9B8L9n3NV1BA%2FpCBH1PuTO4aYWxu2cOhMaDF6beZAmss9hdTX%2Bhon8dNzmDeed%2BKJuLpz%2Buak%2BSTao8G4Hnzu5aKJCaZHiVt30MOraIt6H00VPKO8rzgPl1kJxuJm4NdAra3hQGONyRkujhyzytgXZCyY7Y155TXjjoDavWkj0TxqgiW6S%2BtqJVpiSlQ2kRwYvdmZfhEoj1Izl9q2EQJjKyzFgRWPKG3XSqg0wK6C6%2By2jVcvqOxNcX4jGP5M7alm0vn622KxCdn37hX0O327u6nE3eT9oendtikvN%2F%2BXJlE8wsdmgyAY6ngGQWArnfdpBqSGFS0tQn1uk8V%2Fmknrk

## Step 11: Download Results

If successful, download the results.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step7-post-workitem/

In [16]:
if last_status == "success":
    out_dir = Path.cwd() / "files" / "output"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_zip = out_dir / "output.rvt"
    output_file.download_to(output_path=str(out_zip), token=token)
    print(f"✅ Download successful: {out_zip}")
else:
    print(f"❌ Work item failed or did not complete successfully. Status: {last_status}")

✅ Download successful: c:\Users\aleja\Documents\aps_automation_sdk\examples\03_structural_elements\files\output\output.rvt


## Step 12: Cleanup Resources

Delete the activity and app bundle to clean up resources.

In [17]:
print("Cleaning up: deleting activity and app bundle...")

try:
    delete_activity(activityId=activity_name, token=token)
    print(f"✅ Activity deleted: {activity_full_alias}")
except Exception as e:
    print(f"⚠️ Failed to delete activity: {e}")

try:
    delete_appbundle(appbundleId=app_bundle_name, token=token)
    print(f"✅ App bundle deleted: {app_bundle_name}")
except Exception as e:
    print(f"⚠️ Failed to delete app bundle: {e}")

print("\n🎉 Structural elements workflow completed!")

Cleaning up: deleting activity and app bundle...
✅ Activity deleted: viktortest.CreateBeamElementsActivity+prod
✅ App bundle deleted: CreateBeamElements

🎉 Structural elements workflow completed!
